### Evaluating the m5 limits ###

Create values to be used in `rubin_sim.utils.sysEngVals`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import syseng_throughputs as st

In [2]:
pd.set_option('display.precision', 3)

In [3]:
# Read bandpass information.
defaultDirs = st.setDefaultDirs()
addLosses = True
# Use the X=1.0 atmosphere!
atmosphere = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
hardware, system = st.buildHardwareAndSystem(defaultDirs, addLosses, atmosphereOverride=atmosphere)

In [4]:
# --note these are 1x30s and the overview paper uses 2x15s!!! 
# (this is used as the 'base exposure time' in sysEngVals and the m5 calculation from Cm/dCm_infinity)
# Note also this is using fiducial (old) values of the seeing to set the m5 values as well. 
# thus the resulting values are different than those reported in pstn-054, where the Cm/dCm reported values are 
# for the median delivered seeing in the opsim databases.

m5_std = st.makeM5(hardware, system, readnoise=8.8, othernoise=0, darkcurrent=0.2)

m5_std

,nexp,exptime,FWHMeff,FWHMgeom,skyMag,skyCounts,Zp_t,Tb,Sb,kAtm,gamma,Cm,dCm_infinity,dCm_double,m5,sourceCounts,m5_fid,m5_min
u,1.0,30.0,0.92,0.808,23.052,45.303,26.524,0.023,0.035,0.470,0.038,22.968,0.543,0.344,23.697,405.543,23.9,23.4
g,2.0,15.0,0.87,0.767,22.254,463.634,28.508,0.143,0.173,0.213,0.039,24.512,0.159,0.083,24.902,830.797,25.0,24.6
r,2.0,15.0,0.83,0.734,21.198,988.627,28.361,0.124,0.140,0.126,0.039,24.565,0.081,0.040,24.479,1071.593,24.7,24.3
i,2.0,15.0,0.80,0.710,20.463,1588.281,28.171,0.105,0.114,0.096,0.039,24.517,0.052,0.024,24.103,1271.704,24.0,23.6
z,2.0,15.0,0.78,0.693,19.606,2381.945,27.782,0.073,0.078,0.068,0.039,24.354,0.035,0.016,23.540,1492.761,23.3,22.9
y,2.0,15.0,0.76,0.677,18.602,2717.112,26.818,0.030,0.035,0.171,0.039,23.825,0.031,0.014,22.537,1546.915,22.1,21.7


In [5]:
# Write out values of dictionaries that can be pasted into sims_utils
output_cols = ['Zp_t', 'Tb', 'gamma', 'kAtm', 'Cm', 'dCm_infinity', 'dCm_double', 'skyMag']
for column in output_cols:
    result = '{'
    for filtername, val in zip(m5_std.index, m5_std[column]):
        result += '"%s": %f, ' % (filtername, val)
    result += '}'
    result = result.replace(', }', '}')
    result = 'self.'+column+ ' = ' + result
    print(result)

self.Zp_t = {"u": 26.524237, "g": 28.508375, "r": 28.360838, "i": 28.171396, "z": 27.782264, "y": 26.817819}
self.Tb = {"u": 0.022928, "g": 0.142565, "r": 0.124451, "i": 0.104526, "z": 0.073041, "y": 0.030046}
self.gamma = {"u": 0.037534, "g": 0.038796, "r": 0.039067, "i": 0.039214, "z": 0.039330, "y": 0.039354}
self.kAtm = {"u": 0.470116, "g": 0.212949, "r": 0.126369, "i": 0.095764, "z": 0.068417, "y": 0.171009}
self.Cm = {"u": 22.967681, "g": 24.511574, "r": 24.564727, "i": 24.516816, "z": 24.354430, "y": 23.825489}
self.dCm_infinity = {"u": 0.543325, "g": 0.159045, "r": 0.080846, "i": 0.051845, "z": 0.035176, "y": 0.030969}
self.dCm_double = {"u": 0.343781, "g": 0.083120, "r": 0.039552, "i": 0.024281, "z": 0.015802, "y": 0.013657}
self.skyMag = {"u": 23.051983, "g": 22.253839, "r": 21.197579, "i": 20.462795, "z": 19.606305, "y": 18.601512}
